# ***Pandas***

<sub>Acknowledgement: This notebook summarizing basics of pandas was provided by Hyun Min Kang, an Associate Professor of BioStatistics at the School of Public Health, University of Michigan, Ann Arbor.</sub>

### Creating a data frame

* Creating a dataframe can be done from a dictionary, or a list of list

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'x':[1,2,3],'y':[3,2,1],'s':['a','b','c']})
print(df1)
df2 = pd.DataFrame([[1,3,'a'],[2,2,'b'],[3,1,'c']],columns=['x','y','s'])
print(df2)

### Reading data from a local file

* `pandas` provides `read_csv` function to read comma-separated or tab-separated files
* `.head() function prints first few rows only
* `.columns` attribute list all columns
* '.describe()' function provides a basic summary of each column

In [ ]:
!head sample_data/tips.csv

In [ ]:
import pandas as pd

# Read the file
df = pd.read_csv("sample_data/tips.csv")

# print the very beginning
print(df.head(5))

In [ ]:
# print the column name
for i, v in enumerate(df.columns):  
  print(i,":",v) # print the column index  name
  
# how big is the data?
print("Size:",df.shape)

# get the basic statistics of each column
print(df.describe())

### Reading data online

* You can also read files online using a URL

In [ ]:
import pandas as pd

# Obtain the URL to download the raw data
url = "https://raw.githubusercontent.com/jennybc/gapminder/master/data-raw/08_gap-every-five-years.tsv"
gap = pd.read_csv(url, sep='\t')  # for tsv file, use sep='\t'
print(gap.head(n=5))  # print the beginning of the data

### Exploring pandas dataframe

* `.sample()` function randomly sample rows
* Columns can be accessed as if it is an attribute, or using '[]' like dictionary

In [ ]:
# Get a random sample of the data
print(gap.sample(n=5))

# Accessing column can be done like an attribute
print(gap.country[11:13])

# Or accessed like a dictionary
print(gap['country'][11:13])

### Asking basic questions about the data

* `.unique()` or `.describe()` function can answer some exploratory questions

In [ ]:
# How many countries exist?
print(len(gap.country.unique()))

# How many years exist?
print(len(gap.year.unique()))

# What are available continents?
print(gap.continent.unique())

# Get the basic summary of Life Expectancy
print(gap.lifeExp.describe())

### Subsetting and slicing data

* You may subset or slice specific rows by condition

In [ ]:
# I want to extract only USA data
gap_usa = gap[gap.country == 'United States']
print(gap_usa.shape)
print(gap_usa)

In [ ]:
# I want to extract only 1952 and 2007 data
gap_55yr = gap[gap.year.isin([1952,2007])]
print(gap_55yr.shape)
print(gap_55yr.sample(n=5))

### Reshaping from tall to wide data

* `.pivot()` function allows conversion from tall to wide

In [ ]:
gap_diff = gap_55yr.pivot(index='country',columns='year',values='lifeExp')
gap_diff.reset_index(inplace=True)  # make country as column name again



In [ ]:
# define a new column 'Diff'
gap_diff['Diff'] = gap_diff[2007] - gap_diff[1952]
# define a new column 'Ratio'
gap_diff['Ratio'] = gap_diff[2007]/gap_diff[1952]

gap_diff.columns=['country','1952','2007','Diff','Ratio'] # rename columns

In [ ]:
# print random rows
print(gap_diff.sample(n=5))

In [ ]:
print(gap_diff.describe())

### Changing from wide to tall data

* `.melt()` function allows conversion from wide to tall data.

In [ ]:
gap_diff_t = gap_diff.melt(id_vars = ['country'],  # columns used as ID
                           value_vars = ['Diff','Ratio'], # columns to melt
                           var_name = 'type',      # column names becomes variable
                           value_name = 'value')   # values in columns becomes value
print(gap_diff_t.sample(n=5))

### Concatenating and merging dataframes

* You may concatenate two dataframes by rows or by columns (if consistently ordered)
* You may use `.merge()` function to combine the data when overlapping objects are not ordered in the same way

In [ ]:
# I want to extract only 1952 and 2007 data
gap_1952 = gap[gap.year == 1952]
gap_2007 = gap[gap.year == 2007]

gap_55yr_v2 = pd.concat([gap_1952,gap_2007],axis=0) # combine rows

print(gap_55yr_v2.sample(n=5))
print(gap_55yr.shape)
print(gap_55yr_v2.shape)

In [ ]:
# merge the two data horizontally
gap_55yr_v3 = gap_1952.merge(gap_2007, on = ['country','continent'], suffixes=['_1952','_2007'])

# print the column names of merged data
print(gap_55yr_v3.columns)

# print a peek of the contents
print(gap_55yr_v3.sample(n=5))

### Grouping, aggregating and sorting

* Use `.groupby()` function to create a new DataFrame aggregating on specific attributes
* `.sort_values()` can sort a DataFrame by specific columns

In [ ]:
# group by country, and get avg values
gap_grp = gap.groupby(by=['country','continent']).mean()
print(gap_grp.head(n=10))

In [ ]:
# sort by avg life expectancy
gap_grp.sort_values('lifeExp',ascending=False,inplace=True)
print(gap_grp.head(n=10))

### Very basic plotting with pandas

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/jennybc/gapminder/master/data-raw/08_gap-every-five-years.tsv"
gap = pd.read_csv(url, sep='\t')  # for tsv file, use sep='\t'

In [ ]:
# get the histogram of life Expectancy in 2007
gap_2007 = gap[gap.year == 2007]
gap_2007.lifeExp.plot.hist(bins=20)

In [ ]:
# Visualize how life expectancy changed in US since 1952
gap_usa = gap[gap.country == 'United States']
gap_usa.plot.scatter(x='year',y='lifeExp')